In [13]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf

In [14]:
path = 'lung_colon_image_set/lung_image_sets'

IMG_SIZE = 256
EPOCHS = 10
BATCH_SIZE = 64
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='int',
    batch_size = BATCH_SIZE,
    image_size = (IMG_SIZE, IMG_SIZE),
    validation_split = 0.2, 
    subset = 'training',
    seed = SEED,
    shuffle = True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode= 'int',
    batch_size = BATCH_SIZE,
    image_size = (256, 256),
    validation_split=0.2,
    subset='validation',
    seed = SEED,
    shuffle = True
)

class_names = train_ds.class_names
print(class_names)

Found 15000 files belonging to 3 classes.
Using 12000 files for training.
Found 15000 files belonging to 3 classes.
Using 3000 files for validation.
['lung_aca', 'lung_n', 'lung_scc']


In [15]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
print(len(base_model.layers))

In [ ]:
base_model.trainable = False

In [17]:
from tensorflow.keras import layers, models

In [18]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Dense(3, activation='softmax')
        ])

In [19]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [20]:
def add_preprocess(x, y):
    x = tf.cast(x, tf.float32)
    return preprocess_input(x), y


train_ds = (train_ds.map(add_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .prefetch(tf.data.AUTOTUNE))
val_ds = (val_ds.map(add_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
          .prefetch(tf.data.AUTOTUNE))

In [21]:
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [26]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint

tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)

# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if logs.get('val_accuracy') > 0.90:
#             print('\n Validation accuracy has reached upto 90% so, stopping further training')
#             self.model.stop_training = True

es = EarlyStopping(
    monitor = 'val_loss', 
    restore_best_weights=True
)

ler = ReduceLROnPlateau(
    monitor = 'val_loss',
    patience=2, 
    factor=0.5, 
    verbose=1
)

mc = ModelCheckpoint(
    "best.keras",
    save_best_only=True,
    monitor='val_loss'
)

callbacks = [
    es,            
    mc,            
    # myCallback(),  
    ler 
 ]

In [27]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 761s 4s/step - accuracy: 0.9881 - loss: 0.0318 - val_accuracy: 0.9733 - val_loss: 0.0649 - learning_rate: 0.0010
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 753s 4s/step - accuracy: 0.9931 - loss: 0.0225 - val_accuracy: 0.9913 - val_loss: 0.0215 - learning_rate: 0.0010
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 745s 4s/step - accuracy: 0.9929 - loss: 0.0208 - val_accuracy: 0.9873 - val_loss: 0.0303 - learning_rate: 0.0010


In [28]:
plt.figure(figsize=(8,5))
plt.plot(history.history["val_loss"], label='Validation loss')
plt.plot(history.history['loss'], label='Train loss')
plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.legend()
plt.show()

NameError: name 'history' is not defined

<Figure size 800x500 with 0 Axes>

Fine Tuning

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:100]:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # smaller LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history_finetune.history["val_loss"], label='Validation loss')
plt.plot(history_finetune.history['loss'], label='Train loss')
plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.legend()
plt.show()